In [1]:
import cv2
import math
import numpy as np
from object_detection import ObjectDetection

od = ObjectDetection()

cap = cv2.VideoCapture("traffic.mp4")
classes = od.load_class_names()

count = 0
tracking_objects = {}
track_id = 0
pfcp   = []

try : 
    while True:

        ret,frame = cap.read()

        count += 1

        if not ret:
            break   

        cfcp = [] # current frame center points

        (class_ids,scores,boxes) = od.detect(frame)

        for class_id,box in zip(class_ids,boxes):
            (x,y,w,h) = box
            cx = int(x + (w/2))
            cy = int(y + (h/2))
            cfcp.append((cx,cy))
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(frame,classes[class_id],(x, y - 10), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255), 2)

        if count <= 2:
            for pt2 in cfcp:
                for pt1 in pfcp:
                    distance = math.hypot(pt2[0]-pt1[0],pt2[1]-pt1[1])
                    if distance < 20:
                        tracking_objects[track_id] = pt2
                        track_id += 1
        else:
            tracking_objects_copy = tracking_objects.copy()
            cfcp_copy = cfcp.copy()
            for obj_id,pt1 in tracking_objects_copy.items():
                object_exists = False
                for pt2 in cfcp_copy:
                    distance = math.hypot(pt2[0]-pt1[0],pt2[1]-pt1[1])
                    if distance < 20:
                        tracking_objects[obj_id] = pt2
                        object_exists = True
                        if pt2 in cfcp:
                            cfcp.remove(pt2)
                        break

                if not object_exists:
                    tracking_objects.pop(obj_id)
            
            for pt in cfcp:
                tracking_objects[track_id] = pt
                track_id +=1
                
 
        for obj_id,pt in tracking_objects.items()  :
            cv2.circle(frame,pt,5,(0,0,255),-1)
            cv2.putText(frame,str(obj_id),(pt[0],pt[1]-7),cv2.FONT_HERSHEY_DUPLEX, 0.3, (0, 0, 255), 1)

        cv2.imshow("Frame",frame)
        if cv2.waitKey(1) == ord("n"):
            break

        pfcp = cfcp.copy() # previous frame center points


        
    cap.release()
    cv2.destroyAllWindows()
    
except Exception as e:
    print('Error Occured')
    cap.release()
    cv2.destroyAllWindows()

Loading Object Detection
Running opencv dnn with YOLOv4
